# Nuestro Primer Proceso
Nuestro primer ejemplo será un coche proceso. El coche va a conducir de forma alternativa y estacionar por un tiempo. Cuando se inicia la conducción (o de estacionamiento), se imprimirá el tiempo de simulación actual.

In [1]:
import simpy #importamos nuestra libreria

In [2]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)
env = simpy.Environment()
env.process(car(env))
env.run(until=30)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
Start driving at 19
Start parking at 21
Start driving at 26
Start parking at 28


# Interacción de procesos
La Processinstancia que se devuelve Environment.process() puede ser utilizado para las interacciones de procesos. Los dos ejemplos más comunes de esto son que esperar a otro proceso para terminar y para interrumpir otro proceso mientras se está a la espera de un evento.

# A la espera de un Proceso

In [3]:
class Car(object):
     def __init__(self, env):
         self.env = env
         # Iniciar el proceso de ejecución cada vez que se crea una instancia.
         self.action = env.process(self.run())

     def run(self):
         while True:
             print('Comience estacionamiento y carga en %d' % self.env.now)
             charge_duration = 5
             try:
                 yield self.env.process(self.charge(charge_duration))
             except simpy.Interrupt:
                 # When we received an interrupt, we stop charging and
                 # switch to the "driving" state
                 print('Fue interrumpido. La esperanza, la batería es lo suficientemente completa')


             # The charge process has finished and
             # we can start driving again.
             print('comenzar a conducir en %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)

     def charge(self, duration):
         yield self.env.timeout(duration)
env = simpy.Environment()
car = Car(env)
env.run(until=30)

Comience estacionamiento y carga en 0
comenzar a conducir en 5
Comience estacionamiento y carga en 7
comenzar a conducir en 12
Comience estacionamiento y carga en 14
comenzar a conducir en 19
Comience estacionamiento y carga en 21
comenzar a conducir en 26
Comience estacionamiento y carga en 28


# La interrupción de otro proceso

Simpy le permite interrumpir un proceso que se ejecuta llamando a su interrupt() , El driver proceso tiene una referencia a del coche action proceso. Después de esperar 3 pasos de tiempo, se interrumpe el proceso.

In [4]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=30)

Comience estacionamiento y carga en 0
Fue interrumpido. La esperanza, la batería es lo suficientemente completa
comenzar a conducir en 3
Comience estacionamiento y carga en 5
comenzar a conducir en 10
Comience estacionamiento y carga en 12
comenzar a conducir en 17
Comience estacionamiento y carga en 19
comenzar a conducir en 24
Comience estacionamiento y carga en 26


# Recursos compartidos


# Uso de Recursos básicos

In [5]:
def car(env, name, bcs, driving_time, charge_duration):
     # Simulate driving to the BCS
     yield env.timeout(driving_time)

     # Request one of its charging spots
     print('%s llega a las %d' % (name, env.now))
     with bcs.request() as req:
         yield req

         # Charge the battery
         print('%s comenzando a cobrar a las %s' % (name, env.now))
         yield env.timeout(charge_duration)
         print('%s dejando la BCS a las %s' % (name, env.now))
        
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)

for i in range(4):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))
    
env.run()

Car 0 llega a las 0
Car 0 comenzando a cobrar a las 0
Car 1 llega a las 2
Car 1 comenzando a cobrar a las 2
Car 2 llega a las 4
Car 0 dejando la BCS a las 5
Car 2 comenzando a cobrar a las 5
Car 3 llega a las 6
Car 1 dejando la BCS a las 7
Car 3 comenzando a cobrar a las 7
Car 2 dejando la BCS a las 10
Car 3 dejando la BCS a las 12
